In [1]:
import requests
import pandas as pd
from urllib.parse import urlparse, parse_qs
import numpy as np
import os
import re
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from underthesea import sent_tokenize, word_tokenize
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, T5ForConditionalGeneration, T5Tokenizer
from pyvi import ViPosTagger, ViTokenizer
import gensim

d:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
#Hàm lấy mã sản phẩm từ URL được nhập
def get_product_id(sendo_url):
    # Extract the product ID from the URL
    parsed_url = urlparse(sendo_url)
    # Split the path by '-'
    path_parts = parsed_url.path.split('-')
    # Get the last part of the path, remove '.html', and replace 'p' with ''
    product_id = path_parts[-1].replace('.html', '').replace('p', '')
    return product_id

#Hàm tạo ra cấu trúc để lưu dữ liệu sau khi crawl data
def comment_parser(json):
    d = dict()
    d['id'] = json.get('rating_id')
    d['title'] = json.get('comment_title')
    d['comment'] = json.get('comment')
    d['default_sentiment'] = json.get('status')
    d['like_count'] = json.get('like_count')
    d['customer_id'] = json.get('customer_id')
    d['rating_star'] = json.get('star')
    d['customer_name'] = json.get('user_name')
    return d

#Hàm lấy comment 
def get_comments(product_id):
    """Fetches and parses comments for a given Tiki.vn product ID, including all pages, dropping duplicates."""

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'Accept': '*/*',
        'Accept-Language': 'vi,vi-VN;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
        'Connection': 'keep-alive',
    }

    params = {
        'page': 1,
        'limit': 10,  # Adjust limit as needed
        'sort': 'review_score',
        'v': '2',
        'star': 'all'
    }

    result = []
    while True:
        response = requests.get(f'https://ratingapi.sendo.vn/product/{product_id}/rating',headers=headers,params=params)
        if response.status_code == 200:
            data = response.json().get('data')
            if not data:  # Check if there are no more comments
                break
            for comment in data:
                parsed_comment = comment_parser(comment)
                # Check if comment ID already exists in results before adding
                if parsed_comment['id'] not in [c['id'] for c in result]:
                    result.append(parsed_comment)
            params['page'] += 1
        else:
            print(f"Error getting comments for page {params['page']}. Status code: {response.status_code}")
            break

    df_comment = pd.DataFrame(result)
    return df_comment

#Hàm standardize_comment để chuẩn hóa comment trước khi Sentiment Analysis:
def standardize_comment(comment):
    comment = comment.replace('\n', ' ')\
                    .replace('\r', ' ')\
                    .replace('"', ' ').replace("”", " ")\
                    .replace(":", " ")\
                    .replace("!", " ")\
                    .replace("?", " ") \
                    .replace("-", " ")\
                    .replace(". .", ' ')\
                    .lower()
    return comment


#Hàm xóa Emoji ra khỏi comment
def demoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

# Hàm để tokenize từng comment bằng thư viện underthesea
def tokenize_comment(comment):
    # Tách câu
    sentences = sent_tokenize(comment)
    # Tách từ trong mỗi câu và lưu kết quả
    tokenized_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        # Xóa dấu câu khỏi mỗi từ
        for i, word in enumerate(words):
            words[i] = re.sub(r'[^\w\s]', '', word)

        tokenized_sentences.append(words)
    return tokenized_sentences

# Khởi tạo mô hình và tokenizer 
checkpoint = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

#Hàm sentiment bằng PhoBert cho từng dòng trong cột comment:
def get_sentiment_scores_by_phobert(text, batch_size=32):

    
    num_batches = len(text) // batch_size + (1 if len(text) % batch_size != 0 else 0)
    
    for i in tqdm(range(num_batches), desc="Getting Sentiments..."):
        batch_texts = text[i*batch_size:(i+1)*batch_size]
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
        
        # Get predictions and find dominant sentiment
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        
        for prediction in predictions:
            scores = prediction.tolist()
            highest_score_index = scores.index(max(scores))
            label_mapping = model.config.id2label
            dominant_sentiment = label_mapping[highest_score_index]
    
    return dominant_sentiment


#Hàm dùng ViTokenzier để xử lý phục vụ cho việc feature extraction
def process_comments(df, column_name):
    # dùng hàm ViTokenzier của thư viện pyvi để tách câu trong đoạn, từ trong câu và nối các từ thành một từ ghép có nghĩa
    processed_texts = []
    for comment in df[column_name]:
        # Preprocess với gensim
        lines = gensim.utils.simple_preprocess(str(comment))
        # Ghép lại thành chuỗi
        lines = ' '.join(lines)
        # Tokenize với ViTokenizer
        lines = ViTokenizer.tokenize(lines)
        # Thêm vào danh sách kết quả
        processed_texts.append(lines)
    
    return processed_texts


stopword = open('vietnamese-stopwords.txt',encoding='utf-8').read()
stopword_dash = open('vietnamese-stopwords-dash.txt',encoding='utf-8').read()
# Tạo hàm loại bỏ stopwords
def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
        elif word not in stopword_dash:
            words.append(word)
    return ' '.join(words)

In [25]:
#Lấy commment từ 1 URL
link = input('Nhập vào URL Sendo: ')
product_id = get_product_id(link)

In [26]:
#Lấy comment từ 1 product_id
df = get_comments(product_id)

#Xử lý xóa duplicate và emojis
df = df.drop_duplicates(subset='id', keep='first') # Xóa duplicates dựa trên cột 'id'
df['comment'] = df['comment'].apply(demoji) #Xóa emoji
df['comment'] = df['comment'].apply(standardize_comment)

In [5]:
#Copy dataframe df vào df_comment
df_comment = df.copy()
df_comment['comment'] = df_comment['comment'].apply(str)

#Tokennize cột 'comment' để tạo ra cột mới 'split_tokenized_comment' -> output là các mảng
df_comment['split_tokenized_comment'] = df_comment['comment'].apply(tokenize_comment)

#Tách các mảng sau khi được tokenize thành từng dòng riêng biệt
df_comment = df_comment.explode('split_tokenized_comment')

# Xóa các dòng rỗng trong cột 'split_tokenized_comment'
df_comment.dropna(subset=['split_tokenized_comment'], inplace=True)

# Ghép các mảng trong cột 'split_tokenized_comment' thành text theo từng dòng
df_comment['split_tokenized_comment'] = df_comment['split_tokenized_comment'].apply(lambda x: ' '.join(x))

#Sentiment Analysis với thư viện phoBert, kết quả lưu vào cột mới 'phobert_sentiment_score'
df_comment['phobert_sentiment']= df_comment['split_tokenized_comment'].apply(get_sentiment_scores_by_phobert)

# Tạo cột 'viTokenized_comment' bằng cách áp dụng hàm process_comments cho cột 'split_tokenized_comment'
df_comment['viTokenized_comment'] = process_comments(df_comment, 'split_tokenized_comment')

# Áp dụng hàm remove_stopwords cho cột 'viTokenized_comment'
df_comment['viTokenized_comment'] = df_comment['viTokenized_comment'].apply(remove_stopwords)

# In DataFrame để kiểm tra kết quả
df_comment

Processing Batches: 100%|██████████| 1/1 [00:00<00:00, 25.07it/s]


,id,title,comment,default_sentiment,like_count,customer_id,rating_star,customer_name,split_tokenized_comment,phobert_sentiment,viTokenized_comment
1,16346962,Tuyệt vời,"chuyên nghiệp, thân thiện. đẹp như mô tả. đóng...",Tốt,0,2014246146,5,Trâm Nguyễn,chuyên nghiệp thân thiện,Tích cực,chuyên_nghiệp thân_thiện
1,16346962,Tuyệt vời,"chuyên nghiệp, thân thiện. đẹp như mô tả. đóng...",Tốt,0,2014246146,5,Trâm Nguyễn,đẹp như mô tả,Tích cực,đẹp mô_tả
1,16346962,Tuyệt vời,"chuyên nghiệp, thân thiện. đẹp như mô tả. đóng...",Tốt,0,2014246146,5,Trâm Nguyễn,đóng gói kỹ lưỡng,Tích cực,đóng_gói kỹ_lưỡng
1,16346962,Tuyệt vời,"chuyên nghiệp, thân thiện. đẹp như mô tả. đóng...",Tốt,0,2014246146,5,Trâm Nguyễn,chất lượng tuyệt vời,Tích cực,chất_lượng tuyệt_vời
1,16346962,Tuyệt vời,"chuyên nghiệp, thân thiện. đẹp như mô tả. đóng...",Tốt,0,2014246146,5,Trâm Nguyễn,giá tốt,Tích cực,
...,...,...,...,...,...,...,...,...,...,...,...
611,9898962,Tuyệt vời,chất lượng sản phẩm/dịch vụ rất tốt. gửi sản p...,Tốt,0,2013029596,5,khien ngo,gửi sản phẩm vé dịch vụ nhanh,Tích cực,gửi sản_phẩm vé dịch_vụ
612,9890629,Tuyệt vời,sản phẩm/dịch vụ giống mô tả và tốt hơn mong đ...,Tốt,0,2030115334,5,yến hoàng thị,sản phẩm dịch vụ giống mô tả và tốt hơn mong ...,Tiêu cực,sản_phẩm dịch_vụ mô_tả mong_đợi
612,9890629,Tuyệt vời,sản phẩm/dịch vụ giống mô tả và tốt hơn mong đ...,Tốt,0,2030115334,5,yến hoàng thị,chất lượng sản phẩm dịch vụ rất tốt,Tích cực,chất_lượng sản_phẩm dịch_vụ
613,9852014,Tuyệt vời,sản phẩm/dịch vụ giống mô tả và tốt hơn mong đ...,Tốt,0,2016215846,5,Hà Hải Yến,sản phẩm dịch vụ giống mô tả và tốt hơn mong ...,Tiêu cực,sản_phẩm dịch_vụ mô_tả mong_đợi


In [ ]:
# Tạo bar_chart với 3 màu cho từng bar
removedup = df_comment.copy()
removedup = removedup.drop_duplicates(subset='id', keep='first') #xóa duplicate để tính số lượng comment chính xác

removedup['phobert_sentiment'].value_counts().plot(kind='bar', color=['blue', 'red', 'grey'])
# Gắn nhãn trục x, trục y, tên biểu đồ
plt.xlabel("Loại cảm xúc")
plt.ylabel("Số lượng")
plt.title("Biểu đồ biểu diễn số lượng của từng loại cảm xúc của dữ liệu", color='r')

# Gắn data label cho từng bar

for i, value in enumerate(removedup['phobert_sentiment'].value_counts()):
  plt.text(i, value + 0.1, str(value), ha='center', va='bottom', fontsize=12)

# Hiện chart
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add a grid for better readability
plt.tight_layout()
plt.show()


In [36]:
from nltk import ngrams
def features_list(string, n=1):
    gram_str = list(ngrams(string.split(), n))
    return [ " ".join(gram).lower() for gram in gram_str ]

X = df_comment.viTokenized_comment.apply(lambda t: features_list(t, 1))
df_comment["features"] = X

In [37]:
df_comment_pos = df_comment.loc[df_comment['phobert_sentiment'] == 'Tích cực']
df_comment_neg = df_comment.loc[df_comment['phobert_sentiment'] == 'Tiêu cực']
df_comment_neu = df_comment.loc[df_comment['phobert_sentiment'] == 'Trung tính']

In [38]:
from gensim.models import Word2Vec
def extract_top_adjectives_for_keyword(df, keyword,tag='A', column_name='viTokenized_comment', top_n=10):
    # Tìm các từ liên quan đến từ khóa
    model_word2vec = Word2Vec(df.features.to_list(), vector_size = 100, window = 5, min_count = 1, workers = 4, sg = 1)
    similar_words = model_word2vec.wv.similar_by_word(keyword)
    
    # Lấy các từ liên quan
    related_words = [word for word, score in similar_words]
    
    # Lưu trữ các POS tags của các từ liên quan
    pos_tags = []
    for comment in df[column_name]:
        pos = ViPosTagger.postagging(str(comment))
        pos_tags.append(pos)

    # Lọc ra các từ liên quan với tag chỉ định
    filtered_words = []
    for pos in pos_tags:
        words, tags = pos
        for word, tag_ in zip(words, tags):
            if tag_ == tag and word in related_words:  # Kiểm tra tag và từ liên quan
                filtered_words.append(word)

    # Đếm số lần xuất hiện của các từ
    word_count = {}
    for word in filtered_words:
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1

    # Lấy top N từ phổ biến nhất
    top_words = sorted(word_count, key=word_count.get, reverse=True)[:top_n]
    
    return top_words

In [39]:
model_word2vec.wv.key_to_index

{'chất_lượng': 0,
 'sản_phẩm': 1,
 'dịch_vụ': 2,
 'đẹp': 3,
 'mô_tả': 4,
 'tuyệt_vời': 5,
 'gửi': 6,
 'vé': 7,
 'hàng': 8,
 'đóng_gói': 9,
 'shop': 10,
 'giao': 11,
 'kỹ_lưỡng': 12,
 'nhiệt_tình': 13,
 'tư_vấn': 14,
 'cực_kỳ': 15,
 'hướng_dẫn': 16,
 'chuyên_nghiệp': 17,
 'bất_ngờ': 18,
 'thân_thiện': 19,
 'hợp_lý': 20,
 'ok': 21,
 'kỹ': 22,
 'inox': 23,
 'đc': 24,
 'phục_vụ': 25,
 'mua': 26,
 'bền': 27,
 'ủng_hộ': 28,
 'ko': 29,
 'sử_dụng': 30,
 'vòi': 31,
 'sài': 32,
 'chua': 33,
 'đánh_giá': 34,
 'hài_lòng': 35,
 'tặng': 36,
 'tuy_nhiên': 37,
 'dung': 38,
 'xịt': 39,
 'kg': 40,
 'chờ': 41,
 'mong_đợi': 42,
 'nhẹ': 43,
 'tot': 44,
 'được_cái': 45,
 'pham': 46,
 'dây': 47,
 'xấu': 48,
 'good': 49,
 'hề': 50,
 'đợi': 51,
 'mẫu_mã': 52,
 'vừa_phải': 53,
 'tia': 54,
 'chắc_chắn': 55,
 'thanhk': 56,
 'sét': 57,
 'rỉ': 58,
 'chỗ': 59,
 'cung_ung': 60,
 'ăn_mòn': 61,
 'xây': 62,
 'quyết_định': 63,
 'so_sánh': 64,
 'dep': 65,
 'voi': 66,
 'sen': 67,
 'rat': 68,
 'cam': 69,
 'tưởng': 70,
 'tốt

In [40]:
top_n_adjectives = extract_top_adjectives_for_keyword(df_comment_pos, 'sản_phẩm',tag='A', column_name='viTokenized_comment', top_n=10)
print(top_n_adjectives)

2024-05-24 01:30:11,513 : INFO : collecting all words and their counts
2024-05-24 01:30:11,514 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-05-24 01:30:11,515 : INFO : collected 106 word types from a corpus of 2347 raw words and 976 sentences
2024-05-24 01:30:11,516 : INFO : Creating a fresh vocabulary
2024-05-24 01:30:11,517 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 106 unique words (100.00% of original 106, drops 0)', 'datetime': '2024-05-24T01:30:11.517348', 'gensim': '4.3.2', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'prepare_vocab'}
2024-05-24 01:30:11,518 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 2347 word corpus (100.00% of original 2347, drops 0)', 'datetime': '2024-05-24T01:30:11.518347', 'gensim': '4.3.2', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 

['nhiệt_tình', 'bất_ngờ', 'bền']


In [41]:
top_n_adjectives = extract_top_adjectives_for_keyword(df_comment_pos, 'chất_lượng',tag='A', column_name='viTokenized_comment', top_n=10)
print(top_n_adjectives)

2024-05-24 01:30:13,354 : INFO : collecting all words and their counts
2024-05-24 01:30:13,355 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-05-24 01:30:13,356 : INFO : collected 106 word types from a corpus of 2347 raw words and 976 sentences
2024-05-24 01:30:13,356 : INFO : Creating a fresh vocabulary
2024-05-24 01:30:13,357 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 106 unique words (100.00% of original 106, drops 0)', 'datetime': '2024-05-24T01:30:13.357856', 'gensim': '4.3.2', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'prepare_vocab'}
2024-05-24 01:30:13,358 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 2347 word corpus (100.00% of original 2347, drops 0)', 'datetime': '2024-05-24T01:30:13.358841', 'gensim': '4.3.2', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 

['bền']


In [42]:
top_n_adjectives = extract_top_adjectives_for_keyword(df_comment_neg, 'dịch_vụ',tag='A', column_name='viTokenized_comment', top_n=10)
print(top_n_adjectives)

2024-05-24 01:30:15,261 : INFO : collecting all words and their counts
2024-05-24 01:30:15,262 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-05-24 01:30:15,262 : INFO : collected 60 word types from a corpus of 566 raw words and 145 sentences
2024-05-24 01:30:15,263 : INFO : Creating a fresh vocabulary
2024-05-24 01:30:15,263 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 60 unique words (100.00% of original 60, drops 0)', 'datetime': '2024-05-24T01:30:15.263258', 'gensim': '4.3.2', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'prepare_vocab'}
2024-05-24 01:30:15,264 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=1 leaves 566 word corpus (100.00% of original 566, drops 0)', 'datetime': '2024-05-24T01:30:15.264255', 'gensim': '4.3.2', 'python': '3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit

['tệ']
